# References:

* manual:  
    * [data-download](https://gisgeography.com/how-to-download-sentinel-satellite-data/)  
    * [ndvi-calculation-qgis](https://gisgeography.com/how-to-ndvi-maps-arcgis/)  
    * [image-classification-qgis](https://gisgeography.com/image-classification-techniques-remote-sensing/)  
    
    
* programmatically:  
    * [data-download](http://sentinelsat.readthedocs.io/en/stable/)  
    * [ndvi-calculation](http://neondataskills.org/HDF5/calc-ndvi-python/)  
    * [CCCI](https://www.researchgate.net/publication/259360047_Use_of_the_Canopy_Chlorophyl_Content_Index_CCCI_for_Remote_Estimation_of_Wheat_Nitrogen_Content_in_Rainfed_Environments), [EVI](https://en.wikipedia.org/wiki/Enhanced_vegetation_index), [SAVI](https://en.wikipedia.org/wiki/Soil-adjusted_vegetation_index) 
    * simple solution based on color band wavelength thresholding
    * machine learning based solution:
        * deep learning for computer vision: convolutional neural networks
        * [Kaggle-solution-satellite-images-classification](http://blog.kaggle.com/2017/05/09/dstl-satellite-imagery-competition-3rd-place-winners-interview-vladimir-sergey/)
    
    
    
# Solution:
    * backend: Python Flask + ML lib: scikit-learn/TensorFlow 
    * frontend: ReactJS?

# Data analysis starter code
## Data collection using ESA sentinel data products
References:
* [sentinelsat github](https://github.com/sentinelsat/sentinelsat/blob/127619f6baede1b5cc852b208d4e57e9f4d518ee/README.rst)

In [156]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from geojson import Polygon
from datetime import date
import json
import folium

with open('sentinel_user_login.json', 'r') as fp:
    LOGIN_INFO = json.load(fp)
USER_NAME, PASSWORD = list(LOGIN_INFO.values())
# USER_NAME, PASSWORD = '<YourUserName>', '<YourPassword>'

# define a map polygon
coordinates = [[
    (18.025318309126252, 48.25365988314252),
    (17.945005267006067, 48.18825958525136),
    (18.043165651819628, 48.18825958525136),
    (18.025318309126252, 48.25365988314252),
    (18.025318309126252, 48.25365988314252)
]]
geojson = Polygon(coordinates=coordinates)

# connect to the API
api = SentinelAPI(USER_NAME, PASSWORD, 'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(geojson)
date = (date(2017, 6, 10), date(2017, 7, 24))

# June to July maps
products = api.query(footprint,
                     date=date,
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 40))

product_id = products.keys()[0]  # June

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
geo_df = api.to_geodataframe(products).loc[[product_id]][['geometry']]

def add_choropleth(mapobj, gdf, fill_color='Blue', fill_opacity=0.6,
                   line_opacity=0.2, num_classes=5):
    # Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    # Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON,
    # the data as the GeoDataFrame, the columns as the user-specified id field and and value field.
    # key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(
        geo_data=gdf_wgs84.to_json(),
        fill_color=fill_color,
        fill_opacity=fill_opacity,
        line_opacity=line_opacity,  
    )
    return mapobj

slovakia_map = folium.Map(np.mean(coordinates[0], 0).tolist()[::-1], zoom_start=8)
slovakia_map = add_choropleth(slovakia_map, geo_df)
slovakia_map

In [76]:
# download all results from the search
# api.download_all(products)
api.download(id=product_id, directory_path='../data')

Downloading: 100%|██████████| 328M/328M [02:57<00:00, 3.43MB/s] 
MD5 checksumming: 100%|██████████| 328M/328M [00:00<00:00, 430MB/s] 


{'date': datetime.datetime(2017, 7, 20, 9, 5, 59, 27000),
 'downloaded_bytes': 328128680,
 'footprint': 'POLYGON((18.224327492290943 9.032226956191858,18.22408663416593 9.031170951441243,18.190288805135168 8.882687380398918,18.15652836885776 8.734123505868032,18.122834244765087 8.585483924887761,18.089129232281447 8.436847202629654,18.055507322880683 8.288196640945996,18.02187062054346 8.139610307979558,17.999846132497346 8.04236279416387,17.72146757415633 8.044545576909323,17.728512282037876 9.03659015317746,18.224327492290943 9.032226956191858))',
 'id': u'32dda06c-1fe5-4432-b8fc-fe7d53c4d4b6',
 u'md5': u'23DE96A2F93CBE3CB6FEB374C1783FB6',
 'path': u'../data/S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.zip',
 'size': 328128680,
 'title': u'S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114',
 'url': u"https://scihub.copernicus.eu/dhus/odata/v1/Products('32dda06c-1fe5-4432-b8fc-fe7d53c4d4b6')/$value"}

In [154]:
ll -h ../data/

total 313M
-rw-r--r-- 1 adrian 313M 11 apr 16:43 S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.zip


## Data input types

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

def extract_table(wiki_page, header_row=0):
    soup = BeautifulSoup(requests.get(wiki_page).text, "html5lib")
    table = soup.find("table")
    rows = table.find_all("tr")

    header = [re.sub(r'(\[\d\])', '', x.text) for x in rows[header_row].find_all('th')]
    table = [[x.text for x in row.find_all('td')] for row in rows[header_row+1:]]
    return pd.DataFrame(table, columns=header)

In [8]:
page = 'https://gisgeography.com/how-to-download-sentinel-satellite-data'
wavelength_bands = extract_table(page, header_row=0)
wavelength_bands

,Band,Resolution,Central Wavelength,Description
0,B1,60 m,443 nm,Ultra blue (Coastal and Aerosol)
1,B2,10 m,490 nm,Blue
2,B3,10 m,560 nm,Green
3,B4,10 m,665 nm,Red
4,B5,20 m,705 nm,Visible and Near Infrared (VNIR)
5,B6,20 m,740 nm,Visible and Near Infrared (VNIR)
6,B7,20 m,783 nm,Visible and Near Infrared (VNIR)
7,B8,10 m,842 nm,Visible and Near Infrared (VNIR)
8,B8a,20 m,865 nm,Visible and Near Infrared (VNIR)
9,B9,60 m,940 nm,Short Wave Infrared (SWIR)


## Data input

In [157]:
from zipfile import ZipFile
directory = '../data/S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.zip'
zip_file = ZipFile(directory)

_is_wave_band_img = lambda x: 'B' in x.filename and 'IMG_DATA' in x.filename
band_files = [x for x in zip_file.infolist() if _is_wave_band_img(x)]
zip_file.extractall(members=band_files, path='../data/')

In [159]:
ll -h ../data/S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.SAFE/GRANULE/L1C_T33PZK_A001935_20170720T092114/IMG_DATA/

total 292M
-rw-r--r-- 1 adrian 1,4M 11 apr 17:32 T33PZK_20170720T090559_B01.jp2
-rw-r--r-- 1 adrian  36M 11 apr 17:32 T33PZK_20170720T090559_B02.jp2
-rw-r--r-- 1 adrian  38M 11 apr 17:32 T33PZK_20170720T090559_B03.jp2
-rw-r--r-- 1 adrian  40M 11 apr 17:32 T33PZK_20170720T090559_B04.jp2
-rw-r--r-- 1 adrian  12M 11 apr 17:32 T33PZK_20170720T090559_B05.jp2
-rw-r--r-- 1 adrian  13M 11 apr 17:32 T33PZK_20170720T090559_B06.jp2
-rw-r--r-- 1 adrian  14M 11 apr 17:32 T33PZK_20170720T090559_B07.jp2
-rw-r--r-- 1 adrian  48M 11 apr 17:32 T33PZK_20170720T090559_B08.jp2
-rw-r--r-- 1 adrian 1,3M 11 apr 17:32 T33PZK_20170720T090559_B09.jp2
-rw-r--r-- 1 adrian 701K 11 apr 17:32 T33PZK_20170720T090559_B10.jp2
-rw-r--r-- 1 adrian  13M 11 apr 17:32 T33PZK_20170720T090559_B11.jp2
-rw-r--r-- 1 adrian  13M 11 apr 17:32 T33PZK_20170720T090559_B12.jp2
-rw-r--r-- 1 adrian  14M 11 apr 17:32 T33PZK_20170720T090559_B8A.jp2
-rw-r--r-- 1 adrian  52M 11 apr 17:32 T33PZK_20170720T090559_TCI.jp2


# TODO: convert images from .jp2 to .tiff format
* useful links - needs GDAL:
    * [github](https://github.com/dairejpwalsh/Sentinel-Scripts)
    * [SO](https://gis.stackexchange.com/questions/214489/handle-jp2-sentinel-data)